# bioRSP module

In [1]:
import numpy as np

In [2]:
def find_foreground_background_points(
    gene_name, dge_matrix, tsne_results, dbscan_df, threshold=1, selected_clusters=None
):
    """
    Function to find foreground and background points based on gene expression levels in BioRSP,
    with DBSCAN results integrated to allow for cluster-based analysis. Both foreground and background
    points are filtered by clusters if specified.

    Parameters:
    - gene_name: The gene of interest.
    - dge_matrix: A dataframe containing the gene expression data (rows = genes, columns = cells).
    - tsne_results: A 2D numpy array with the t-SNE (or UMAP) coordinates for each cell.
    - dbscan_df: DataFrame with DBSCAN cluster labels for each cell.
    - threshold: The expression level threshold for the foreground points. Default is 1.
    - selected_clusters: A list of cluster labels to focus on, or None to include all cells.

    Returns:
    - foreground_points: A list of (x, y) coordinates for cells with gene expression above the threshold.
    - background_points: A list of (x, y) coordinates for all cells (or cells in selected clusters).
    """

    dbscan_clusters = dbscan_df["Cluster"].values

    if len(dbscan_clusters) != tsne_results.shape[0]:
        raise ValueError(
            "DBSCAN cluster labels do not match the number of t-SNE results."
        )

    if gene_name not in dge_matrix.index:
        raise ValueError(f"Gene {gene_name} not found in the dataset.")

    gene_expression = dge_matrix.loc[gene_name]
    cell_barcodes = dge_matrix.columns
    cell_index_map = {barcode: idx for idx, barcode in enumerate(cell_barcodes)}

    # Foreground points: cells where gene expression is above the threshold
    foreground_indices = gene_expression[gene_expression > threshold].index
    foreground_points = [
        tsne_results[cell_index_map[barcode]] for barcode in foreground_indices
    ]
    foreground_points = np.array(foreground_points)

    if selected_clusters is not None:
        foreground_indices_filtered = [
            i
            for i in foreground_indices
            if dbscan_clusters[cell_index_map[i]] in selected_clusters
        ]
        foreground_points = [
            tsne_results[cell_index_map[barcode]]
            for barcode in foreground_indices_filtered
        ]

        # Filter cells that are part of the selected clusters for background points
        selected_indices = [
            i
            for i, cluster in enumerate(dbscan_clusters)
            if cluster in selected_clusters
        ]
        background_points = tsne_results[selected_indices]
    else:
        background_points = tsne_results

    return np.array(foreground_points), np.array(background_points)

In [3]:
def convert_to_polar(coords, vantage_point):
    translated_coords = coords - vantage_point

    r = np.sqrt(translated_coords[:, 0] ** 2 + translated_coords[:, 1] ** 2)
    theta = np.arctan2(translated_coords[:, 1], translated_coords[:, 0])
    theta = np.mod(theta + 2 * np.pi, 2 * np.pi)

    sorted_indices = np.argsort(theta)
    sorted_r = r[sorted_indices]
    sorted_theta = theta[sorted_indices]

    return sorted_r, sorted_theta

In [ ]:
def in_scanning_range(pt_theta, angle, window):
    angular_difference = np.abs((pt_theta - angle + np.pi) % (2 * np.pi) - np.pi)
    return angular_difference <= window / 2

In [ ]:
def compute_histogram(projection, resolution, angle, window):
    start_angle = (angle - window / 2) % (2 * np.pi)
    end_angle = (angle + window / 2) % (2 * np.pi)

    if start_angle > end_angle:
        proportion_before_wrap = (2 * np.pi - start_angle) / window

        bins_before_wrap = max(1, int(np.floor(proportion_before_wrap * resolution)))
        bins_after_wrap = max(1, resolution - bins_before_wrap)

        histogram1, _ = np.histogram(
            projection, bins=bins_before_wrap, range=(start_angle, 2 * np.pi)
        )
        histogram2, _ = np.histogram(
            projection, bins=bins_after_wrap, range=(0, end_angle)
        )

        histogram = np.concatenate((histogram1, histogram2))
    else:
        histogram, _ = np.histogram(
            projection, bins=resolution, range=(start_angle, end_angle)
        )

    return histogram

In [ ]:
def compute_cdf(histogram):
    cdf = np.cumsum(histogram).astype(np.float64) / np.sum(histogram)
    return cdf


def compute_cdfs(
    fg_projection, bg_projection, angle, scanning_window, resolution, mode
):
    # Both foreground and background projections present
    if fg_projection.shape[0] > 0 and bg_projection.shape[0] > 0:
        fg_histogram = compute_histogram(
            fg_projection, resolution, angle, scanning_window
        )
        bg_histogram = compute_histogram(
            bg_projection, resolution, angle, scanning_window
        )

        fg_cdf = compute_cdf(fg_histogram)
        bg_cdf = compute_cdf(bg_histogram)

        if mode == "absolute":
            fg_cdf *= fg_projection.shape[0] / bg_projection.shape[0]

        return fg_cdf, bg_cdf

    # No foreground projections present
    if bg_projection.shape[0] > 0:
        bg_histogram = compute_histogram(
            bg_projection, resolution, angle, scanning_window
        )

        bg_cdf = compute_cdf(bg_histogram)

        return np.zeros(resolution), bg_cdf

    # No data present
    return np.zeros(resolution), np.zeros(resolution)

In [ ]:
def compute_area(fg_cdf, bg_cdf):
    area_diff = np.trapz(bg_cdf - fg_cdf, dx=1 / fg_cdf.shape[0])
    return area_diff


def calculate_differences(
    fg_points,
    bg_points,
    scanning_window,
    resolution,
    vantage_point,
    angle_range,
    mode,
):
    # Convert to polar coordinates and sort
    _, fg_theta = convert_to_polar(fg_points, vantage_point)
    _, bg_theta = convert_to_polar(bg_points, vantage_point)

    # Calculate CDFs for each angle
    differences = np.empty(resolution)

    for i, angle in enumerate(np.linspace(angle_range[0], angle_range[1], resolution)):
        fg_in_range = in_scanning_range(fg_theta, angle, scanning_window)
        bg_in_range = in_scanning_range(bg_theta, angle, scanning_window)

        fg_projection = fg_theta[fg_in_range]
        bg_projection = bg_theta[bg_in_range]

        fg_cdf, bg_cdf = compute_cdfs(
            fg_projection, bg_projection, angle, scanning_window, resolution, mode
        )
        differences[i] = compute_area(fg_cdf, bg_cdf)

    return differences


def calculate_rsp_area(
    fg_points,
    bg_points,
    vantage_point,
    scanning_window=np.pi,
    resolution=1000,
    angle_range=np.array([0, 2 * np.pi]),
    mode="absolute",
):
    """
    Calculate the RSP area and RMSD.

    Returns:
        - rsp_area (float): Calculated RSP area.
        - differences (ndarray): Differences calculated during the process.
        - rmsd (float): Root Mean Square Deviation.
    """
    differences = calculate_differences(
        fg_points,
        bg_points,
        scanning_window,
        resolution,
        vantage_point,
        angle_range,
        mode,
    )

    segment_areas = 0.5 * (2 * np.pi / resolution) * np.power(differences, 2)
    rsp_area = np.sum(segment_areas)

    # RMSD calculation
    rmsd = np.sqrt(np.mean(np.square(differences)))

    return rsp_area, differences, rmsd

In [ ]:
def calculate_deviation_score(rsp_area, differences, resolution):
    """
    Calculate the deviation score.

    Parameters:
        - rsp_area (float): Calculated RSP area.
        - differences (ndarray): Differences calculated during the process.
        - resolution (int): Resolution for the calculation.

    Returns:
        - deviation_score (float): Deviation score.
    """
    # Area of a circle with the same area as RSP area
    radius = np.sqrt(rsp_area / np.pi)

    # Calculate the overlap area
    intersection_area = np.sum(
        0.5 * (2 * np.pi / resolution) * np.power(np.minimum(differences, radius), 2)
    )

    # Deviation score
    deviation_score = 1 - intersection_area / rsp_area

    return deviation_score

In [ ]:
def rsp(
    fg_points,
    bg_points,
    vantage_point,
    scanning_window=np.pi,
    resolution=1000,
    angle_range=np.array([0, 2 * np.pi]),
    mode="absolute",
):
    """
    Perform the full RSP analysis including RSP area, RMSD, and deviation score.

    Returns:
        - rsp_area (float): Calculated RSP area.
        - rmsd (float): Root Mean Square Deviation.
        - deviation_score (float): Deviation score.
    """
    rsp_area, differences, rmsd = calculate_rsp_area(
        fg_points,
        bg_points,
        vantage_point,
        scanning_window,
        resolution,
        angle_range,
        mode,
    )

    deviation_score = calculate_deviation_score(rsp_area, differences, resolution)

    return rsp_area, rmsd, deviation_score